In [1]:
#data loading
import pandas as pd
predata = pd.read_csv(r'C:/Users/BeatrizCarvalho/OneDrive - Closer Consultoria Lda/Documents/Entangled-Spaces/Transfers/Datasets/2transfers_balanced_smote+rund-1M.csv', low_memory = False)

In [2]:
#-------------------------------------------------pre-processing--------------------------------------------------------------#

#begin of preprocessing
import time

start = time.time()

In [2]:
#convert amount and accountbalance to classes and assign a word to each interval  
import numpy as np

#automatic labels
import string


class LabelCategorizer:
    def __init__(self, base_word='cat'):
        self.initial = 1
        self._alphabet_index = 0
        self.base_word = base_word
        self.current_word = self.base_word
        self.shift = 0

    def __str__(self):
        return 'Class: Label Categorizer\nBase word: ' + self.base_word + '\nCurrent Word: ' + self.current_word

    def get_next_word(self):
        if self.initial:
            self.initial = 0
            return self.current_word

        if self.shift > 0:
            self.current_word = self.current_word[-1] + self.current_word[:-1]
            self.shift -= 1
        else:
            self.current_word = self.current_word + string.ascii_lowercase[self._alphabet_index]
            self._alphabet_index = (self._alphabet_index + 1) % len(string.ascii_lowercase)
            self.shift = len(self.current_word) - 1

        return self.current_word


#replacement of the old columns with the new ones with classes
def cutter(col, number, word, words_map):
    #make sure that only positives are assigned an interval
    col_min = max(predata[col].min(), 1)
    col_max = max(predata[col].max(), 1)

    bins_a = np.geomspace(float(col_min), float(col_max), num=number)
    bins_a[0] = bins_a[0] - 1
    bins_aux = bins_a[1:]
    bins_aux = np.append(bins_aux, bins_a[-1] + 1)
    bin_tuples = list(zip(bins_a, bins_aux))

    bins = pd.IntervalIndex.from_tuples(bin_tuples)

    #range of the intervals made
    labels_a = []

    a = LabelCategorizer(base_word=word)

    for _ in range(number):
        labels_a.append(a.get_next_word())

    x = pd.cut(predata[col].to_list(), bins=bins)
    x.categories = labels_a
    predata[col] = x

    for i in range(number):
        words_map[labels_a[i]] = bins[i]

    #columns to apply the conversion


columns = ['amount', 'accountbalance']

#number of intervals for each column
number_bins = [36, 38]

#base words assigned to each column on columns to apply the conversion
base_words = ['pink', 'red']

#get acess to the range of the interval based on the word that appears
values_map = {}

for i in range(len(columns)):
    cutter(columns[i], number_bins[i], base_words[i], values_map)

In [4]:
#bins amount
#transfers: 10m - 25.123 / 1M - 35.8775 / 2M - 37.3285 / 3M - 38.2194 / 4M - 38.818 / 5M - 39.3326 / gans - 26.3159 / imbalanced - 24.7233
#payments: 10m -  / 1M - 36.7452 / 2M - 38.2134 / 3M - 39.1709 / 4M - 39.8368 / 5M - 40.2234 / gans - 24.8089 / imbalanced - 22.5533

#bins accountbalance
#transfers: 10m - 24.6482 / 1M - 37.0555 / 2M - 38.5003 / 3M - 39.4439 / 4M - 40.1224 / 5M - 40.627 / gans - 25.734 / imbalanced - 23.9742
#payments: 10m - / 1M - 37.234 / 2M - 38.6645 / 3M - 39.651 / 4M - 40.2506 / 5M - 40.7402 / gans - 26.0935 / imbalanced - 20.7986

In [3]:
#assign the word negaccount for negative values of accountbalance  
aux = predata['accountbalance'].values

vacc = []

for elm in aux:
    if str(elm) == 'nan':
        vacc.append('negaccount')
    else:
        vacc.append(elm)
predata['accountbalance'] = vacc

In [9]:
#interval that a word corresponds to
#values_map['apbicndke']

Interval(34055.72399772817, 50124.37964687015, closed='right')

In [4]:
#convert hours to classes and assign a word to each interval  
bins_hour = [0, 4, 8, 12, 16, 20, 24]

#labels assigned to each interval
labels_hour = ['dawn', 'earlymorning', 'morning', 'afternoon', 'dusk', 'night']

predata['hour'] = list(pd.cut(predata['hour'], bins=bins_hour, labels=labels_hour, retbins=True, include_lowest=True)[0])

In [5]:
#map trusted_indicator values to words
ti_value_map = {
    '0.0': 'ntrusted',
    '0': 'ntrusted',
    '1.0': 'trusted',
    'unknown': 'tunknown'
}

predata['trusted_indicator'] = predata['trusted_indicator'].apply(lambda x: ti_value_map[x])

In [6]:
#add letter before number to distinguish between similar numbers from different columns
cols = ['entity', 'reference', 'iban_orig', 'iban_dest', 'ipaddress', 'clientid', 'week']

identifier = ['e', 'r', 'io', 'id', 'ip', 'c', 'w']

for col in range(len(cols)):
    predata[cols[col]] = predata[cols[col]].apply(lambda x: identifier[col] + str(x))

In [7]:
#convert binary and chains of numbers to specific words
def apply_map(df, col, target_map):
    df[col] = df[col].apply(lambda x: target_map.get(str(x)))


cols_maps = [('is_fraud', {'0': 'nfraud', '1': 'fraud'}),
             ('weekday', {'0': 'mon', '1': 'tue', '2': 'wed', '3': 'thu', '4': 'fri', '5': 'sat', '6': 'sun'}),
             ('month', {'1': 'jan', '2': 'feb', '3': 'mar', '4': 'apr', '5': 'may', '6': 'jun', '7': 'jul', '8': 'aug', 
                        '9': 'sep', '10': 'oct', '11': 'nov', '12': 'dec'})]

for comb in cols_maps:
    apply_map(predata, comb[0], comb[1])

In [8]:
#make fraud column as the center column
new_order = ['canal', 'operativa', 'clientid', 'entity', 'reference', 'trusted_indicator', 'iban_orig', 'iban_dest',
             'amount', 'is_fraud', 'accountbalance', 'ipaddress', 'browser_family', 'os_family', 'hour', 'week', 'weekday',
             'month', 'device']

predata = predata[new_order]

In [9]:
#select data for train and test  
from sklearn.model_selection import train_test_split

#target column
y = predata['is_fraud']
predata.drop('is_fraud', axis=1)

#train and test
X_train, X_test, y_train, y_test = train_test_split(predata, y, stratify=y, test_size=0.3)

In [10]:
#function for conversion format
def convert_to_w2v_format(df):
    sentences = df.to_numpy()
    sentences_aux = [list(curr) for curr in sentences]
    sentences_series = pd.Series(sentences_aux)
    return sentences_series

In [11]:
#for entering the model
sentences_series = convert_to_w2v_format(X_train)

In [12]:
#for the test metrics
X_test_np = X_test.copy()
sentences_series_np_test = convert_to_w2v_format(X_test_np)

In [13]:
#for the train metrics
X_train_np = X_train.copy()
sentences_series_np_train = convert_to_w2v_format(X_train_np)

In [14]:
#get size of the corpus 
token_count = sum([len(sentence) for sentence in sentences_series])

print("This corpus contains {} tokens".format(token_count))

This corpus contains 13300000 tokens


In [15]:
#end of preprocessing
stop = time.time()

print(f"Pre-Processing: {stop - start}s")

Training time: 43.16966795921326s


In [17]:
#-----------------------------------------------------training----------------------------------------------------------------#

#begin of training
begin = time.time()

In [15]:
#inicialization and training word2vec 
import multiprocessing
from myWord2Vec import MyWord2Vec
from gensim.models.callbacks import CallbackAny2Vec

def training(sentences, dim, sample, negative, alpha, min_alpha):
    model = MyWord2Vec(
        sg = 1,  #skip-gram 
        workers = multiprocessing.cpu_count(),  #use all cores 
        vector_size = dim,  #dimension of the embedding space 
        window = 1,  #words befores and after the center word 
        sample = sample,  #whithout subsampling 
        min_count = 1,  #use every word 
        negative = negative,  #noise-words 
        hs = 0,  #negative sampling
        ns_exponent = 0.75,  #exponent to shape negative sampling 
        alpha = alpha,  #initial learning rate 
        min_alpha = min_alpha,  #final learning rate 
    )

    #vocabulary creation
    model.build_vocab(sentences)

    #model training
    model.train(sentences, epochs = 150, total_examples = model.corpus_count, compute_loss = True, callbacks = [])

    return model

In [19]:
#end of training
end = time.time()

print(f"Training: {end - begin}s")

Training: 6.508042335510254s


In [24]:
#-----------------------------------------------------prediction-------------------------------------------------------------#

#begin of test - 1st step
begin = time.time()

In [16]:
#update probv - vector with the parameters of a transaction
from scipy.special import softmax

def predictTransaction(my_model, m_cid, my_curr_cid, verbose):
    #obtain the representative vector of the clientid
    civ = my_model.wv.get_vector(m_cid)

    #multiplication of the clientid vector by the decode matrix (M2)  
    m2 = my_model.syn1neg
    
    #vector of len = vocab_size that softmax is applied to obtain the conditional probabilities 
    vout = softmax(np.dot(civ, m2.T))

    fraud_curr = my_curr_cid.copy()
    fraud_curr.append(my_model.wv.index_to_key.index('fraud'))

    nfraud_curr = my_curr_cid.copy()
    nfraud_curr.append(my_model.wv.index_to_key.index('nfraud'))

    #filter the vector to the transaction fields
    fraud_probv = sum(vout[fraud_curr])
    nfraud_probv = sum(vout[nfraud_curr])

    #print results if verbose True
    if verbose:
        print(f"Transaction: {my_curr_cid}\nFraud prob: {fraud_probv}\nNon Fraud prob: {nfraud_probv}")

    #threshold for false negatives
    # if fraud_probv > threshold: return 1

    if fraud_probv > nfraud_probv:
        return 1
    
    return 0

In [17]:
#prediction method   
def predict(model, X, verbose1, verbose2):
    #save the predictions made by the model in a list
    predictions = []

    #for each transaction (eval_row)
    for i, eval_row in enumerate(X, 1):

        #print transaction parameters if verbose1 True
        if verbose1:
            print(eval_row)

        #cid is always in position 2 of the array
        curr_cid = eval_row[2]

        #for each sentence_series creates a current list
        curr = []

        #if curr_cid is not known
        if curr_cid not in model.wv.index_to_key:
            #update model
            model.build_vocab([[curr_cid]], update=True)

        #filter the parameters associated with the transaction made by the specific clientid (civ)
        for x in eval_row:
            if x != curr_cid:
                if x in model.wv.index_to_key:
                    curr.append(model.wv.index_to_key.index(x))

                else:
                    #if the word is not known                        
                    #update model
                    model.build_vocab([[x]], update=True)
                    curr.append(model.wv.index_to_key.index(x))

        #sum each value on the array to obtain the final probability
        curr_prediction = predictTransaction(model, curr_cid, curr, verbose2)
        predictions.append(curr_prediction)

    return predictions

In [ ]:
#end of test - 1st step
end = time.time()

print(f"1st Step Test: {end - begin}s")

In [18]:
#convert true values for the train set to binary
y_train = [1 if elem == "fraud" else 0 for elem in y_train]

In [19]:
#convert true values of the test set to binary
y_test = [1 if elem == "fraud" else 0 for elem in y_test]

In [20]:
#grid-search
import os
from mlflow import log_metric, log_param, log_artifact
import mlflow
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix

#create a hyperparameters dictionary 
dim = [2, 5]   
negative = [5, 10] #0 leads to not having the syn1neg property
alpha = [0.025, 0.035] 
min_alpha = [0.0001, 0.001] 
sample = [0, 0.001, 0.01]

#train the model 
i = 0
for d in dim:
    for n in negative:
        for a in alpha:
            for m in min_alpha:
                for s in sample:
                    i+=1
                    with mlflow.start_run(nested=True):

                        # Log params to mlflow
                        log_param("dim", d)
                        log_param("negative", n)
                        log_param("alpha", a)
                        log_param("min_alpha", m)
                        log_param("sample", s)

                        curr_model = training(sentences_series_np_train, dim = d, sample = s, negative = n, alpha = a, min_alpha = m)

                        #metrics for each combination - train
                        prevs_train = predict(curr_model, sentences_series_np_train[:10000], verbose1 = False, verbose2 = False)
                        precision_train, recall_train, fscore_train, _ = precision_recall_fscore_support(y_train[:10000], prevs_train, average='micro')
                        tn_train, fp_train, fn_train, tp_train = confusion_matrix(y_train[:10000], prevs_train, labels = [0, 1]).ravel()
                        
                        log_param("accuracy_train", accuracy_score(y_train[:10000], prevs_train))
                        log_param("precision_train", precision_train)
                        log_param("recall_train", recall_train)
                        log_param("f1_train", fscore_train)
                        log_param("tn_train", tn_train)
                        log_param("fp_train", fp_train)
                        log_param("fn_train", fn_train)
                        log_param("tp_train", tp_train)

                        #save current model
                        curr_model.save("trained_model.w2v")
                        log_artifact("trained_model.w2v")
                        os.remove("trained_model.w2v")

Validation loss decreased (inf --> 6469611.000000).  Saving model ...
Validation loss decreased (6469611.000000 --> 5823474.000000).  Saving model ...
Validation loss decreased (5823474.000000 --> 5066955.000000).  Saving model ...
Validation loss decreased (5066955.000000 --> 2719172.000000).  Saving model ...
Validation loss decreased (2719172.000000 --> 2642420.000000).  Saving model ...
Validation loss decreased (2642420.000000 --> 2587682.000000).  Saving model ...
Validation loss decreased (2587682.000000 --> 2574880.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
Validation loss decreased (2574880.000000 --> 2008590.000000).  Saving model ...
Validation loss decreased (2008590.000000 --> 1911160.000000).  Saving model ...
Validation loss decreased (1911160.000000 --> 1894028.000000).  Saving model ...
Validation loss decreased (1894028.000000 --> 1805596.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss

EarlyStopping counter: 9 out of 25
EarlyStopping counter: 10 out of 25
Validation loss decreased (1750544.000000 --> 1740768.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (1740768.000000 --> 1521120.000000).  Saving model ...
Validation loss decreased (1521120.000000 --> 30736.000000).  Saving model ...
Validation loss decreased (30736.000000 --> 30536.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out of 25
EarlyStopping counter: 6 out of 25
EarlyStopping counter: 7 out of 25
EarlyStopping counter: 8 out of 25
EarlyStopping counter: 9 out of 25
EarlyStopping counter: 10 out of 25
EarlyStopping counter: 11 out of 25
EarlyStopping counter: 12 out of 25
EarlyStopping counter: 13 out of 25
EarlyStopping counter: 14 out of 25
EarlyStopping counter: 15 out of 25
EarlyStopping counter: 16 out of 25
EarlyStop

EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
Validation loss decreased (2574126.000000 --> 2558540.000000).  Saving model ...
Validation loss decreased (2558540.000000 --> 2065450.000000).  Saving model ...
Validation loss decreased (2065450.000000 --> 1910084.000000).  Saving model ...
Validation loss decreased (1910084.000000 --> 1862900.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (1862900.000000 --> 1829444.000000).  Saving model ...
Validation loss decreased (1829444.000000 --> 1812712.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
Validation loss decreased (1812712.000000 --> 1810956.000000).  Saving model ...
Validation loss decreased (1810956.000000 --> 1794488.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
Va

Validation loss decreased (2062504.000000 --> 1582416.000000).  Saving model ...
Validation loss decreased (1582416.000000 --> 54488.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (54488.000000 --> 52952.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
Validation loss decreased (52952.000000 --> 51272.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out of 25
EarlyStopping counter: 6 out of 25
EarlyStopping counter: 7 out of 25
EarlyStopping counter: 8 out of 25
EarlyStopping counter: 9 out of 25
EarlyStopping counter: 10 out of 25
Validation loss decreased (51272.000000 --> 51256.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping

EarlyStopping counter: 23 out of 25
EarlyStopping counter: 24 out of 25
EarlyStopping counter: 25 out of 25
Validation loss decreased (inf --> 9611386.000000).  Saving model ...
Validation loss decreased (9611386.000000 --> 8148036.000000).  Saving model ...
Validation loss decreased (8148036.000000 --> 4033486.000000).  Saving model ...
Validation loss decreased (4033486.000000 --> 3948928.000000).  Saving model ...
Validation loss decreased (3948928.000000 --> 3884236.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (3884236.000000 --> 2454100.000000).  Saving model ...
Validation loss decreased (2454100.000000 --> 2395268.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out of 25
EarlyStopping counter: 6 out of 25
EarlyStopping counter: 7 out of 25
EarlyStopping counter: 8 out of 25
EarlyStopping counter

...

EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
Validation loss decreased (77896.000000 --> 77072.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
Validation loss decreased (77072.000000 --> 75872.000000).  Saving model ...
Validation loss decreased (75872.000000 --> 74368.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (74368.000000 --> 73888.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (73888.000000 --> 73376.000000).  Saving model ...
Validation loss decreased (73376.000000 --> 73120.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
Validation loss decreased (73120.000000 --> 73104.000000).  Saving model ...
Validation loss decreased (73104.000000 --> 72208.000000).  Saving model ...
EarlyStopping counter: 1 out of 25

EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out of 25
EarlyStopping counter: 6 out of 25
Validation loss decreased (83944.000000 --> 83408.000000).  Saving model ...
Validation loss decreased (83408.000000 --> 81968.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
Validation loss decreased (81968.000000 --> 80640.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
Validation loss decreased (80640.000000 --> 80632.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (80632.000000 --> 80264.000000).  Saving model ...
Validation loss decreased (80264.000000 --> 79784.000000).  Saving model ...
Validation loss decreased (79784.000000 --> 78608.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Valida

EarlyStopping counter: 1 out of 25
Validation loss decreased (53904.000000 --> 53256.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (53256.000000 --> 52584.000000).  Saving model ...
Validation loss decreased (52584.000000 --> 52528.000000).  Saving model ...
Validation loss decreased (52528.000000 --> 51408.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (51408.000000 --> 51016.000000).  Saving model ...
Validation loss decreased (51016.000000 --> 49960.000000).  Saving model ...
Validation loss decreased (49960.000000 --> 49152.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (49152.000000 --> 47952.000000).  Saving model ...
Validation loss decreased (47952.000000 --> 46648.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (46648.000000 --> 46576.000000).  Saving model ...
Validation loss decreased (46576.000000 --> 45704.00000

EarlyStopping counter: 4 out of 25
Validation loss decreased (68640.000000 --> 67936.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out of 25
EarlyStopping counter: 6 out of 25
EarlyStopping counter: 7 out of 25
EarlyStopping counter: 8 out of 25
EarlyStopping counter: 9 out of 25
EarlyStopping counter: 10 out of 25
EarlyStopping counter: 11 out of 25
EarlyStopping counter: 12 out of 25
EarlyStopping counter: 13 out of 25
EarlyStopping counter: 14 out of 25
EarlyStopping counter: 15 out of 25
EarlyStopping counter: 16 out of 25
EarlyStopping counter: 17 out of 25
EarlyStopping counter: 18 out of 25
EarlyStopping counter: 19 out of 25
EarlyStopping counter: 20 out of 25
EarlyStopping counter: 21 out of 25
EarlyStopping counter: 22 out of 25
EarlyStopping counter: 23 out of 25
EarlyStopping counter: 24 out of 25
EarlyStopping counter: 25 out of 

EarlyStopping counter: 6 out of 25
EarlyStopping counter: 7 out of 25
Validation loss decreased (80800.000000 --> 79448.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (79448.000000 --> 77736.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (77736.000000 --> 75512.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
Validation loss decreased (75512.000000 --> 75336.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
Validation loss decreased (75336.000000 --> 72440.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25


In [ ]:
#apply the prediction method for the train set
#predict(model, X_train_np.values, verbose1=True, verbose2=True)

In [ ]:
#true values for the train set
#y_train

In [ ]:
#metrics for the train set
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, matthews_corrcoef, confusion_matrix, precision_recall_curve

trs = y_train
prevs = predict(model, X_train_np.values, verbose1 = False, verbose2 = False, shouldBe = trs)

#accuracy
accuracy = accuracy_score(trs,prevs)
print('accuracy: {}'.format(accuracy))

#precision, recall, f-score
precision, recall, fscore, support = precision_recall_fscore_support(trs, prevs, average = 'micro')

#precision - ratio tp / (tp + fp) - ability not to label a negative sample as positive
print('precision: {}'.format(precision))

#recall - ratio tp / (tp + fn) - ability to find all the positive samples - best is 1, worst is 0
print('recall: {}'.format(recall))

#fscore - weighted harmonic mean of the precision and recall - best is 1, worst is 0
print('fscore: {}'.format(fscore))

#matthews correlation coefficient - measure of the quality of binary classifications
#can be used even if the classes are of very different sizes - is in essence a correlation coefficient between -1 and +1
#+1 means perfect prediction, 0 an average random prediction, -1 an inverse prediction
mcc = matthews_corrcoef(trs, prevs)
print('mcc: {}'.format(mcc))

#g-mean - squared root of the product of the sensitivity and specificity - best is 1, worst is 0
print('G-mean:', (geometric_mean_score(trs, prevs, average = 'micro')))

#sensitivity - ability to predict true positives of each available category = recall

#confusion matrix    
print(confusion_matrix(trs, prevs))

#true positives, false positives, true negatives, false negatives
tn, fp, fn, tp = confusion_matrix(trs,prevs).ravel()

#true negatives
print('true negatives: {}'.format(tn))

#false positives
print('false positives: {}'.format(fp))

#false negatives
print('false negatives: {}'.format(fn))

#false positives
print('true positives: {}'.format(tp))

#specificity - ability to predict true negatives of each available category - recall of the negative class
specifity = tn / (tn + fp)
print('specifity: {}'.format(specifity))

#roc curve
import matplotlib.pyplot as plt
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(trs,prevs)

roc_auc = metrics.auc(fpr, tpr)

display = metrics.RocCurveDisplay(fpr = fpr, tpr = tpr, roc_auc = roc_auc, estimator_name = 'Word2vec')
display.plot()
plt.show()

In [ ]:
#apply the prediction method for the test set
#predict(model, X_test_np.values, verbose1=True, verbose2=True)

In [ ]:
#true values of the test set
#set(y_test)

In [21]:
#load the ideal model
import gensim.models.word2vec as w2v

model = w2v.Word2Vec.load(r'C:/Users/BeatrizCarvalho/OneDrive - Closer Consultoria Lda/Pictures/transfers-1M/trained_model.w2v')

In [22]:
#begin of test - 2nd step
import time

begin = time.time()

In [23]:
#metrics for the test set 
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, matthews_corrcoef, confusion_matrix, precision_recall_curve

trs = y_test[:10000]
prevs = predict(model, X_test_np.values[:10000], verbose1 = False, verbose2 = True)

#accuracy
accuracy = accuracy_score(trs, prevs)
print('accuracy: {}'.format(accuracy))

#precision, recall, f-score
precision, recall, fscore, support = precision_recall_fscore_support(trs, prevs, average = 'micro')

#precision - ratio tp / (tp + fp) - ability not to label a negative sample as positive
print('precision: {}'.format(precision))

#recall - ratio tp / (tp + fn) - ability to find all the positive samples - best is 1, worst is 0
print('recall: {}'.format(recall))

#fscore - weighted harmonic mean of the precision and recall - best is 1, worst is 0
print('fscore: {}'.format(fscore))

#matthews correlation coefficient - measure of the quality of binary classifications
#can be used even if the classes are of very different sizes - is in essence a correlation coefficient between -1 and +1
#+1 means perfect prediction, 0 an average random prediction, -1 an inverse prediction
mcc = matthews_corrcoef(trs, prevs)
print('mcc: {}'.format(mcc))

#g-mean - squared root of the product of the sensitivity and specificity - best is 1, worst is 0
print('G-mean:', (geometric_mean_score(trs, prevs, average = 'micro')))

#sensitivity - ability to predict true positives of each available category = recall

#confusion matrix    
print(confusion_matrix(trs, prevs))

#true positives, false positives, true negatives, false negatives
tn, fp, fn, tp = confusion_matrix(trs, prevs).ravel()

#true negatives
print('true negatives: {}'.format(tn))

#false positives
print('false positives: {}'.format(fp))

#false negatives
print('false negatives: {}'.format(fn))

#false positives
print('true positives: {}'.format(tp))

#specificity - ability to predict true negatives of each available category - recall of the negative class
specifity = tn / (tn + fp)
print('specifity: {}'.format(specifity))

#precision-recall curve - compute precision-recall pairs for different probability thresholds
#print(precision_recall_curve(trs, prevs))

#roc curve
import matplotlib.pyplot as plt
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(trs, prevs)

roc_auc = metrics.auc(fpr, tpr)

display = metrics.RocCurveDisplay(fpr = fpr, tpr = tpr, roc_auc = roc_auc, estimator_name = 'Word2vec')
display.plot()
plt.show()

Transaction: [11, 16, 0, 1, 2, 1630325, 1630326, 37, 9, 53, 180, 5, 4, 6, 93, 10, 21, 3]
Fraud prob: 8.587571208806821e-06
Non Fraud prob: 8.671804401672034e-06
Transaction: [11, 7, 0, 1, 2, 1630328, 1630329, 51, 9, 38, 1630330, 5, 4, 6, 14, 10, 13, 3]
Fraud prob: 1.7440954223957306e-05
Non Fraud prob: 1.7500840414186314e-05
Transaction: [11, 7, 0, 1, 2, 1630332, 191, 41, 9, 43, 120, 5, 19, 6, 57, 25, 21, 23]
Fraud prob: 9.067551822283804e-06
Non Fraud prob: 9.044809623048877e-06
Transaction: [12, 7, 0, 1, 2, 159, 1630333, 36, 9, 38, 161, 5, 4, 6, 14, 26, 21, 3]
Fraud prob: 0.0069684851205640275
Non Fraud prob: 0.006968484823022536
Transaction: [12, 16, 0, 1, 2, 1630335, 1630336, 37, 9, 42, 1630337, 5, 4, 6, 14, 10, 13, 3]
Fraud prob: 2.2190015092959925e-05
Non Fraud prob: 2.2201725803583372e-05
Transaction: [11, 7, 0, 1, 2, 1630339, 1630340, 37, 9, 34, 1630341, 5, 4, 6, 14, 10, 13, 3]
Fraud prob: 1.6093140345674328e-05
Non Fraud prob: 1.6049952932917222e-05
Transaction: [12, 7, 0, 1, 

Transaction: [11, 16, 0, 1, 2, 372609, 1630436, 45, 8, 35, 1630437, 5, 4, 22, 95, 27, 21, 3]
Fraud prob: 4.4842407845635535e-05
Non Fraud prob: 4.4835014477939694e-05
Transaction: [11, 16, 0, 1, 2, 1630439, 1630440, 41, 9, 35, 1630441, 5, 4, 6, 14, 27, 32, 3]
Fraud prob: 1.641107105498918e-05
Non Fraud prob: 1.6433983784480688e-05
Transaction: [12, 7, 0, 1, 2, 1630443, 1630444, 41, 9, 53, 1630445, 5, 4, 6, 14, 31, 13, 3]
Fraud prob: 1.0946553317126018e-05
Non Fraud prob: 1.0953420854775686e-05
Transaction: [111, 16, 0, 1, 2, 173, 178, 115, 9, 96, 110, 30, 33, 20, 87, 10, 13, 23]
Fraud prob: 0.08783275657454534
Non Fraud prob: 0.08783275652855851
Transaction: [11, 7, 0, 1, 2, 1630447, 1630448, 36, 9, 38, 1630449, 5, 4, 6, 14, 10, 13, 3]
Fraud prob: 1.396183085944358e-05
Non Fraud prob: 1.3933003089050544e-05
Transaction: [11, 16, 0, 1, 2, 1630451, 249, 51, 9, 43, 251, 5, 4, 40, 107, 26, 50, 3]
Fraud prob: 1.8597750582216577e-05
Non Fraud prob: 1.8560940944212234e-05
Transaction: [12, 7,

Transaction: [11, 7, 0, 1, 2, 1630538, 1630539, 51, 9, 43, 1630540, 5, 4, 6, 14, 10, 21, 23]
Fraud prob: 1.826166781881966e-05
Non Fraud prob: 1.8221192689793497e-05
Transaction: [11, 15, 0, 1, 17, 44882, 1630541, 69, 8, 168, 1630542, 5, 4, 6, 128, 27, 58, 3]
Fraud prob: 0.00016661201587736405
Non Fraud prob: 0.00016664914633962444
Transaction: [18, 15, 0, 1, 2, 1630544, 1630545, 66, 8, 146, 932202, 28, 19, 6, 108, 25, 60, 29]
Fraud prob: 8.30929522699364e-06
Non Fraud prob: 8.33944509004425e-06
Transaction: [12, 15, 0, 1, 17, 1088, 1630546, 51, 8, 74, 750, 5, 4, 22, 129, 26, 50, 3]
Fraud prob: 0.0005363584610878472
Non Fraud prob: 0.0005363597314299009
Transaction: [18, 15, 0, 1, 2, 111750, 111751, 66, 8, 24, 1630547, 28, 19, 22, 108, 31, 60, 29]
Fraud prob: 0.0005317269141670952
Non Fraud prob: 0.0005318166368710283
Transaction: [11, 16, 0, 1, 2, 102, 101, 63, 9, 233, 18902, 5, 19, 6, 57, 10, 21, 23]
Fraud prob: 0.06902838267215802
Non Fraud prob: 0.06902838266991741
Transaction: [11

Transaction: [12, 7, 0, 1, 2, 1630630, 1630631, 162, 9, 190, 1173, 5, 4, 6, 14, 10, 13, 3]
Fraud prob: 1.0651502790892664e-05
Non Fraud prob: 1.0652004831968043e-05
Transaction: [11, 16, 0, 1, 2, 1630633, 1630634, 63, 8, 89, 1630635, 5, 19, 85, 123, 31, 55, 23]
Fraud prob: 1.1063130244792774e-05
Non Fraud prob: 1.0989429739538537e-05
Transaction: [11, 7, 0, 1, 2, 1630637, 1630638, 37, 9, 42, 1630639, 5, 4, 6, 14, 26, 13, 3]
Fraud prob: 8.777384294944568e-06
Non Fraud prob: 8.840825074685199e-06
Transaction: [12, 15, 0, 1, 2, 91396, 1630640, 44, 8, 34, 47445, 5, 4, 22, 107, 10, 50, 3]
Fraud prob: 0.00016733334372564612
Non Fraud prob: 0.00016735671542311237
Transaction: [11, 7, 0, 1, 2, 1630642, 1167004, 41, 8, 35, 1630643, 5, 4, 40, 109, 68, 39, 3]
Fraud prob: 2.7165505912307708e-05
Non Fraud prob: 2.71367770778852e-05
Transaction: [12, 7, 0, 1, 2, 1630645, 1630646, 36, 9, 38, 1630647, 5, 4, 6, 14, 10, 21, 3]
Fraud prob: 1.371668790284275e-05
Non Fraud prob: 1.3707539977758643e-05
Tran

Transaction: [11, 16, 0, 1, 2, 1630737, 1630738, 44, 9, 49, 180, 5, 4, 6, 93, 10, 21, 3]
Fraud prob: 2.30916022871952e-05
Non Fraud prob: 2.310162236085489e-05
Transaction: [11, 7, 0, 1, 2, 1630739, 169, 41, 9, 59, 157, 5, 4, 40, 77, 10, 39, 3]
Fraud prob: 9.216021436507101e-05
Non Fraud prob: 9.214575709413504e-05
Transaction: [11, 16, 0, 1, 2, 165, 163, 36, 9, 35, 121, 5, 4, 20, 64, 10, 32, 3]
Fraud prob: 0.01598336311204851
Non Fraud prob: 0.015983362812436952
Transaction: [12, 15, 0, 1, 2, 159, 415, 36, 9, 38, 161, 5, 4, 6, 14, 26, 21, 3]
Fraud prob: 0.0049852846551197705
Non Fraud prob: 0.004985284399334877
Transaction: [12, 7, 0, 1, 2, 1630741, 249697, 54, 8, 38, 1630742, 5, 4, 6, 88, 31, 46, 3]
Fraud prob: 1.3046865969812416e-05
Non Fraud prob: 1.3041863635265827e-05
Transaction: [11, 16, 0, 1, 2, 247, 169, 100, 9, 43, 157, 5, 4, 40, 77, 10, 39, 3]
Fraud prob: 0.005976381848223451
Non Fraud prob: 0.005976382076147214
Transaction: [12, 16, 0, 1, 2, 1630744, 1630745, 70, 9, 67, 16

...

Transaction: [11, 16, 0, 1, 17, 1640844, 509, 51, 9, 43, 148, 5, 4, 20, 77, 25, 39, 3]
Fraud prob: 1.1512904694921389e-05
Non Fraud prob: 1.1556626606346043e-05
Transaction: [11, 7, 0, 1, 2, 1640846, 1640847, 70, 9, 53, 262, 5, 4, 6, 57, 25, 21, 3]
Fraud prob: 1.078881373217655e-05
Non Fraud prob: 1.0846939652253695e-05
Transaction: [18, 15, 0, 1, 2, 234099, 234098, 41, 8, 78, 1410, 28, 19, 22, 108, 71, 39, 29]
Fraud prob: 0.000197912247417057
Non Fraud prob: 0.0001979619689168633
Transaction: [12, 7, 0, 1, 17, 1640849, 1640850, 45, 8, 53, 223278, 5, 4, 20, 117, 25, 39, 3]
Fraud prob: 1.837027016904358e-05
Non Fraud prob: 1.836926125520222e-05
Transaction: [11, 16, 0, 1, 2, 381, 382, 51, 9, 43, 92, 5, 4, 40, 62, 10, 21, 3]
Fraud prob: 0.004179436152172156
Non Fraud prob: 0.004179434282349415
Transaction: [12, 7, 0, 1, 17, 1403805, 1403806, 48, 8, 42, 1640852, 5, 4, 40, 123, 25, 55, 3]
Fraud prob: 1.4258470656614008e-05
Non Fraud prob: 1.4229967149503864e-05
Transaction: [18, 7, 0, 1, 2

Transaction: [12, 15, 0, 1, 2, 18377, 1640943, 51, 8, 96, 105378, 5, 4, 20, 132, 25, 60, 3]
Fraud prob: 0.00014450704720501858
Non Fraud prob: 0.0001444678503565849
Transaction: [12, 7, 0, 1, 17, 891670, 1640944, 41, 8, 38, 1640945, 5, 4, 20, 119, 31, 60, 3]
Fraud prob: 7.231075881439683e-05
Non Fraud prob: 7.238803732865126e-05
Transaction: [18, 15, 0, 1, 2, 1640947, 1432204, 91, 8, 24, 1082206, 28, 19, 40, 94, 71, 50, 29]
Fraud prob: 3.3443367414065506e-05
Non Fraud prob: 3.342233182479504e-05
Transaction: [18, 15, 0, 1, 2, 159595, 1640948, 48, 8, 24, 1640949, 28, 19, 40, 87, 25, 13, 29]
Fraud prob: 7.071411802788816e-05
Non Fraud prob: 7.080900390482157e-05
Transaction: [11, 15, 0, 1, 17, 871140, 871141, 41, 8, 34, 560, 5, 4, 20, 95, 10, 50, 3]
Fraud prob: 6.173838670349377e-05
Non Fraud prob: 6.177677526693295e-05
Transaction: [11, 7, 0, 1, 2, 1640951, 1640952, 54, 9, 59, 120, 5, 19, 6, 57, 10, 21, 23]
Fraud prob: 9.005739595835394e-06
Non Fraud prob: 8.965879057853954e-06
Transact

Transaction: [11, 7, 0, 1, 17, 734252, 1641056, 41, 9, 34, 1641057, 5, 4, 6, 88, 10, 46, 3]
Fraud prob: 1.1703197330348303e-05
Non Fraud prob: 1.1715136636780699e-05
Transaction: [12, 7, 0, 1, 2, 1056902, 1641058, 48, 8, 78, 24871, 30, 75, 6, 109, 25, 39, 3]
Fraud prob: 0.00011563144812849127
Non Fraud prob: 0.00011566273531111373
Transaction: [11, 7, 0, 1, 2, 1641060, 1641061, 66, 8, 67, 426528, 5, 4, 20, 106, 25, 21, 3]
Fraud prob: 3.541578394106182e-05
Non Fraud prob: 3.539692477261269e-05
Transaction: [18, 15, 0, 1, 2, 146151, 321705, 61, 8, 24, 146171, 28, 19, 40, 98, 71, 13, 29]
Fraud prob: 0.00020392615034639993
Non Fraud prob: 0.00020398035379346402
Transaction: [18, 15, 0, 1, 2, 1175407, 1641063, 91, 8, 24, 1641064, 28, 19, 85, 134, 68, 58, 29]
Fraud prob: 8.642328308994252e-06
Non Fraud prob: 8.5985343769579e-06
Transaction: [12, 16, 0, 1, 2, 213, 86, 115, 9, 67, 99, 5, 4, 22, 76, 27, 13, 3]
Fraud prob: 0.004231401777413439
Non Fraud prob: 0.004231401621154996
Transaction: [1

Transaction: [12, 7, 0, 1, 2, 1641164, 1641165, 51, 9, 38, 1641166, 5, 4, 6, 14, 10, 13, 3]
Fraud prob: 2.031495995424848e-05
Non Fraud prob: 2.0314019195666333e-05
Transaction: [12, 7, 0, 1, 2, 1087149, 1641167, 70, 9, 49, 1641168, 5, 4, 6, 14, 10, 13, 3]
Fraud prob: 8.970358552019577e-05
Non Fraud prob: 8.96710358701094e-05
Transaction: [12, 7, 0, 1, 2, 1641170, 1641171, 36, 9, 38, 1641172, 5, 4, 6, 14, 10, 13, 3]
Fraud prob: 1.0571767006695154e-05
Non Fraud prob: 1.0575231556231302e-05
Transaction: [12, 7, 0, 1, 2, 699155, 1641173, 45, 8, 53, 28111, 52, 4, 22, 88, 26, 46, 3]
Fraud prob: 0.00010264864990637958
Non Fraud prob: 0.00010261779885133561
Transaction: [18, 7, 0, 1, 17, 1641175, 1641176, 63, 8, 24, 819790, 28, 19, 6, 124, 25, 58, 29]
Fraud prob: 2.246951796536223e-05
Non Fraud prob: 2.2500731937213914e-05
Transaction: [12, 7, 0, 1, 2, 1641178, 1641179, 70, 9, 73, 1641180, 5, 4, 6, 14, 10, 13, 3]
Fraud prob: 1.3630850489221302e-05
Non Fraud prob: 1.3630277820197989e-05
Transa

KeyboardInterrupt: 

In [ ]:
#end of test - 2nd step
end = time.time()

print(f"2nd Step Test: {end - begin}s")

In [64]:
#---------------------------------------------------density matrix------------------------------------------------------------#

#product of the matrices

#weight matrices
m1 = model.wv.vectors
m2 = model.syn1neg  #negative sampling
#m2 = model.syn1    #hierarchical-softmax 

mproduct = np.matmul(m1, m2.T)
mproduct.shape

(21938, 21938)

In [65]:
#apply softmax to obtain a matrix with conditional probabilities
conditional_probs = softmax(mproduct)

In [49]:
#confirm that softmax sum is 1
conditional_probs.sum()

0.99994487

In [50]:
#maximum value of the conditional probabilities
np.max(conditional_probs)

1.8367052e-07

In [51]:
#minimum value of the conditional probabilities
np.min(conditional_probs)

1e-45

In [78]:
#visualize the 1st 20 lines and the last 20 lines of the density matrix

#1st 20 lines
conditional_aux = conditional_probs[:20].copy()

#last 20 lines
conditional_aux = np.concatenate((conditional_aux, conditional_probs[-20:].copy()))

In [79]:
#save density matrix - after the product and the softmax
pd.DataFrame(conditional_aux).to_csv(r'C:/Users/BeatrizCarvalho/OneDrive - Closer Consultoria Lda/Documents/Entangled-Spaces/Transfers/Profiles/density_matrix.csv', header = None, index = False)

In [80]:
#shape of the compressed density matrix 
conditional_aux.shape

(40, 21938)

In [81]:
#-------------------------------------------------eigenvalues-----------------------------------------------------------------#

#singular eingenvalues of the square matrix - after the product and before softmax - w has the eigenvalues and v the eigenvectors
w, v = np.linalg.eig(mproduct)  #mproduct is an array of arrays

#separate real and imaginary parts of the singular eigenvalues
x = w.real  #array

y = w.imag  #array

In [82]:
#save eigenvalues in csv
df = pd.DataFrame({"Real" : x, "Imaginary" : y})

df.to_csv(r'C:/Users/BeatrizCarvalho/OneDrive - Closer Consultoria Lda/Documents/Entangled-Spaces/Transfers/Profiles/eigenvalues.csv', index = False)